Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [1]:
import os
import pandas as pd
import numpy as np
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
pd.options.display.float_format = '{:.3f}'.format

In [2]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
pd.to_datetime(client.time()['serverTime'], unit='ms')
# client.time()

Timestamp('2022-05-08 20:14:08.762000')

If your time is off you need to synchronize with timeserver

In [3]:
pd.json_normalize(client.account()['balances'])

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [4]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]

In [5]:
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols).astype(float)
assets

BTCUSDT                                                    \
          Open time      Open       High       Low     Close   Volume   
0 1651622400000.000 38127.160  67952.010  9000.000 39695.800 3289.479   
1 1651708800000.000 39690.010  48256.090 12000.000 36551.040 3747.859   
2 1651795200000.000 36552.970  90000.000  9500.000 35844.540 5133.879   
3 1651881600000.000 35851.700 179685.000  2000.000 35472.390 2648.748   
4 1651968000000.000 35472.400 172120.000  6988.000 34575.320 5038.611   

                                                         \
         Close Time Quote asset volume Number of trades   
0 1651708799999.000      128626868.064       109403.000   
1 1651795199999.000      142179013.425       136426.000   
2 1651881599999.000      185618082.671       195320.000   
3 1651967999999.000       94693433.702       110202.000   
4 1652054399999.000      174239090.272       184146.000   

                               ... XRPUSDT                           \
  Taker buy base asset volume  ...    High   Low Close       Volume   
0                    2070.214  ...   0.650 0.608 0.646  7188003.900   
1                    2305.807  ...   0.657 0.580 0.598 11682556.700   
2                    2791.128  ...   0.616 0.581 0.581 12824794.100   
3                    1439.332  ...   0.603 0.569 0.582  4378006.600   
4                    2883.387  ...   0.583 0.558 0.572  8803779.100   

                                                         \
         Close Time Quote asset volume Number of trades   
0 1651708799999.000        4497422.006         8638.000   
1 1651795199999.000        7147202.834        13976.000   
2 1651881599999.000        7706389.172        15203.000   
3 1651967999999.000        2579982.368         5475.000   
4 1652054399999.000        5035015.212         9581.000   

                                                                   
  Taker buy base asset volume Taker buy quote asset volume Ignore  
0                 7020708.900                  4393175.815  0.000  
1                11566933.900                  7074742.840  0.000  
2                12610238.800                  7578080.053  0.000  
3                 4097615.100                  2413970.942  0.000  
4                 8522774.000                  4874268.185  0.000  

[5 rows x 72 columns]

Formula to convert from unix time to datetime, needs to be set as index

In [6]:
pd.concat([pd.to_datetime(assets[symbol, 'Open time'], unit='ms') for symbol in symbols], axis=1)

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
,Open time,Open time,Open time,Open time,Open time,Open time
0,2022-05-04,2022-05-04,2022-05-04,2022-05-04,2022-05-04,2022-05-04
1,2022-05-05,2022-05-05,2022-05-05,2022-05-05,2022-05-05,2022-05-05
2,2022-05-06,2022-05-06,2022-05-06,2022-05-06,2022-05-06,2022-05-06
3,2022-05-07,2022-05-07,2022-05-07,2022-05-07,2022-05-07,2022-05-07
4,2022-05-08,2022-05-08,2022-05-08,2022-05-08,2022-05-08,2022-05-08


Swapping levels for easier selection 

In [7]:
assets = assets.swaplevel(axis=1)
assets

,Open time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,Number of trades,Taker buy base asset volume,...,High,Low,Close,Volume,Close Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,...,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT
0,1651622400000.000,38127.160,67952.010,9000.000,39695.800,3289.479,1651708799999.000,128626868.064,109403.000,2070.214,...,0.650,0.608,0.646,7188003.900,1651708799999.000,4497422.006,8638.000,7020708.900,4393175.815,0.000
1,1651708800000.000,39690.010,48256.090,12000.000,36551.040,3747.859,1651795199999.000,142179013.425,136426.000,2305.807,...,0.657,0.580,0.598,11682556.700,1651795199999.000,7147202.834,13976.000,11566933.900,7074742.840,0.000
2,1651795200000.000,36552.970,90000.000,9500.000,35844.540,5133.879,1651881599999.000,185618082.671,195320.000,2791.128,...,0.616,0.581,0.581,12824794.100,1651881599999.000,7706389.172,15203.000,12610238.800,7578080.053,0.000
3,1651881600000.000,35851.700,179685.000,2000.000,35472.390,2648.748,1651967999999.000,94693433.702,110202.000,1439.332,...,0.603,0.569,0.582,4378006.600,1651967999999.000,2579982.368,5475.000,4097615.100,2413970.942,0.000
4,1651968000000.000,35472.400,172120.000,6988.000,34575.320,5038.611,1652054399999.000,174239090.272,184146.000,2883.387,...,0.583,0.558,0.572,8803779.100,1652054399999.000,5035015.212,9581.000,8522774.000,4874268.185,0.000


Returns

In [8]:
close = assets["Close"].copy()

In [9]:
returns = close.pct_change().dropna()

In [10]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
1,-0.079,-0.066,-0.060,-0.089,-0.136,-0.074
2,-0.019,-0.034,-0.585,-0.013,0.096,-0.028
3,-0.010,-0.007,1.327,-0.012,0.022,0.001
4,-0.025,-0.029,-0.018,0.011,0.053,-0.017


In [11]:
def annualised_risk_return(returns):
    summary = returns.agg(["mean", "std"]).T
    summary.columns = ["Return", "Risk"]
    summary.Return = summary.Return*252
    summary.Risk = summary.Risk * np.sqrt(252)
    return summary

In [12]:
summary = annualised_risk_return(returns)

In [13]:
summary

,Return,Risk
BTCUSDT,-8.456,0.493
ETHUSDT,-8.528,0.384
BNBUSDT,41.818,12.951
LTCUSDT,-6.528,0.693
TRXUSDT,2.180,1.606
XRPUSDT,-7.490,0.506
